In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.0 MB/s eta 0:00:00


In [2]:
pip install numpy

In [3]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00


In [4]:
pip install accelerate -U

In [5]:
pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
squad

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5000
})

In [ ]:
squad = squad.train_test_split(test_size=0.2)

In [ ]:
squad["train"][0]

{'id': '56bf6b0f3aeaaa14008c9602',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'In which decade did Beyonce become famous?',
 'answers': {'text': ['late 1990s'], 'answer_start': [276]}}

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(squad["train"])

,id,title,context,question,answers
0,5733b2fe4776f4190066108f,University_of_Notre_Dame,"The Lobund Institute grew out of pioneering research in germ-free-life which began in 1928. This area of research originated in a question posed by Pasteur as to whether animal life was possible without bacteria. Though others had taken up this idea, their research was short lived and inconclusive. Lobund was the first research organization to answer definitively, that such life is possible and that it can be prolonged through generations. But the objective was not merely to answer Pasteur's question but also to produce the germ free animal as a new tool for biological and medical research. This objective was reached and for years Lobund was a unique center for the study and production of germ free animals and for their use in biological and medical investigations. Today the work has spread to other universities. In the beginning it was under the Department of Biology and a program leading to the master's degree accompanied the research program. In the 1940s Lobund achieved independent status as a purely research organization and in 1950 was raised to the status of an Institute. In 1958 it was brought back into the Department of Biology as integral part of that department, but with its own program leading to the degree of PhD in Gnotobiotics.",Work on a germ-free-life ended up in the creation of which Notre Dame institute?,"{'text': ['The Lobund Institute'], 'answer_start': [0]}"
1,56bf9f6aa10cfb14005511d2,Beyoncé,"Her fourth studio album 4 was released on June 28, 2011 in the US. 4 sold 310,000 copies in its first week and debuted atop the Billboard 200 chart, giving Beyoncé her fourth consecutive number-one album in the US. The album was preceded by two of its singles ""Run the World (Girls)"" and ""Best Thing I Never Had"", which both attained moderate success. The fourth single ""Love on Top"" was a commercial success in the US. 4 also produced four other singles; ""Party"", ""Countdown"", ""I Care"" and ""End of Time"". ""Eat, Play, Love"", a cover story written by Beyoncé for Essence that detailed her 2010 career break, won her a writing award from the New York Association of Black Journalists. In late 2011, she took the stage at New York's Roseland Ballroom for four nights of special performances: the 4 Intimate Nights with Beyoncé concerts saw the performance of her 4 album to a standing room only.",Who awarded Beyonce and award for writing?,"{'text': ['New York Association of Black Journalists'], 'answer_start': [640]}"
2,57340a0dd058e614000b6852,Montana,"East of the divide, several roughly parallel ranges cover the southern part of the state, including the Gravelly Range, the Madison Range, Gallatin Range, Absaroka Mountains and the Beartooth Mountains. The Beartooth Plateau is the largest continuous land mass over 10,000 feet (3,000 m) high in the continental United States. It contains the highest point in the state, Granite Peak, 12,799 feet (3,901 m) high. North of these ranges are the Big Belt Mountains, Bridger Mountains, Tobacco Roots, and several island ranges, including the Crazy Mountains and Little Belt Mountains.",What is thie highest point in the state?,"{'text': ['Granite Peak'], 'answer_start': [371]}"
3,573018d5947a6a140053d0e7,Antibiotics,"In empirical therapy, a patient has proven or suspected infection, but the responsible microorganism is not yet unidentified. While the microorgainsim is being identified the doctor will usually administer the best choice of antibiotic that will be most active against the likely cause of infection usually a broad spectrum antibiotic. Empirical therapy is usually initiated before the doctor knows the exact identification of microorgansim causing the infection as the identification process make take several days in the laboratory.",At what stage does a doctor begin empirical therapy?,"{'text': ['While the microorgainsim is being identified'], 'answer_start': [126]}"
4,56d4e8472ccc5

In [15]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, TrainingArguments, Trainer
import torch
from datasets import load_dataset

# Load pre-trained DistilBERT model and tokenizer
model_name = "distilbert-base-cased"
model = DistilBertForQuestionAnswering.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

# Load the SQuAD dataset
squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)["train"]

# Create a test dataset
test_dataset = []
for example in squad:
    question = example["question"]
    context = example["context"]
    answer_text = example["answers"]["text"][0]
    answer_start = example["answers"]["answer_start"][0]
    answer = context[answer_start: answer_start + len(answer_text)]
    test_dataset.append({
        "question": question,
        "context": context,
        "answer": answer,
    })

# Process the test dataset
def truncate_context(context, max_length):
    return context[:max_length]

test_dataset_processed = []
for example in test_dataset:
    question = example["question"]
    context = example["context"]
    answer = example["answer"]
    encoded_inputs = tokenizer(question, truncate_context(context, 512), return_tensors="pt")
    output = model(**encoded_inputs)
    start_logits, end_logits = output.start_logits, output.end_logits
    start_pred = torch.argmax(start_logits)
    end_pred = torch.argmax(end_logits)
    predicted_answer = tokenizer.decode(encoded_inputs["input_ids"][start_pred:end_pred+1], skip_special_tokens=True)
    test_dataset_processed.append({
        "context": context,
        "question": question,
        "answer": answer,
        "predicted_answer": predicted_answer,
    })

# Print the first example in the processed test dataset
print(test_dataset_processed[0])

import time

# Calculate accuracy
correct_predictions = sum(1 for example in test_dataset_processed if example["predicted_answer"] == example["answer"])
accuracy = correct_predictions / len(test_dataset_processed)

# Measure performance time
start_time = time.time()
for example in test_dataset_processed:
    # Process each example
    end_time = time.time()
    performance_time = end_time - start_time

print("Accuracy:", accuracy)
print("Performance time:", performance_time, "seconds")


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'context': 'Apple debuted the iTunes Wi-Fi Music Store on September 5, 2007, in its Media Event entitled "The Beat Goes On...". This service allows users to access the Music Store from either an iPhone or an iPod Touch and download songs directly to the device that can be synced to the user\'s iTunes Library over a WiFi connection, or, in the case of an iPhone, the telephone network.', 'question': 'On what date was the iTunes Wi-Fi Music Store launched?', 'answer': 'September 5, 2007', 'predicted_answer': ''}
Accuracy: 0.0
Performance time: 0.0011837482452392578 seconds
